# utils

> Module of utility functions used throughout the package, for things like common preprocessing steps many models are likely to use, or useful for multiple points of analysis.

In [1]:
#| default_exp loader

In [2]:
#| hide 
%load_ext autoreload
%autoreload 2

In [3]:
#| hide
from nbdev.showdoc import *


In [4]:
#| export

import csv
import os
import time
from enum import Enum, auto
from typing import Dict, List, Optional, Tuple, Union
from pathlib import Path

import numpy as np

## CSV utilities

In [5]:
#| export

def determine_header_rows_and_delimiter(
    filename: Path | str
) -> Tuple[Optional[int], Optional[str]]:
    """
    Given a filename pointing at a CSV files, decides:
     * how many header lines there are (based on first line starting with a digit)
     * the delimiter-- right now tries whitespace and comma

    Returns one of:
     - (number of header rows, column delimiter),
     - (number of header rows, None) if the delimiter could not be inferred,
     - (None, None) if CSV has no numerical rows,

    :param filename: CSV Path or filepath literal
    :return: header and delimiter information, if possible.
    """
    MAX_ROWS = 100  # if you don't have data within first 100 lines, exit.
    header_row_count = 0
    with open(filename) as f:
        header_found = False
        line = ""  # modified in while below

        # Search over lines until we find one starting with a digit
        while not header_found:
            line = f.readline()
            if line == "":  # last line of file reached
                return None, None
            line = line.strip()
            try:
                int(line[0])
                header_found = True
            except ValueError:
                header_row_count += 1
            except IndexError:
                header_row_count += 1

            # guard against infinite loop
            if header_row_count >= MAX_ROWS:
                return None, None

        # Now try splitting that first line of data
        delim_guesses = [" ", ",", ", "]

        for guess in delim_guesses:
            try:
                comps = line.split(guess)  # whitespace separated?
                float(comps[0])
                return header_row_count, guess
            except ValueError:
                continue
        return header_row_count, None


## Activity counts

In [6]:
#| export

class ActivityCountAlgorithm(Enum):
    te_Lindert_et_al = 0
    ActiGraphOfficial = 1
    ADS = 2


def build_activity_counts(
    data,
    axis: int = 3,
    prefix: str = "",
    algorithm: ActivityCountAlgorithm = ActivityCountAlgorithm.ADS
) -> Tuple[np.ndarray, np.ndarray]:
    if algorithm == ActivityCountAlgorithm.ActiGraphOfficial:
        return build_ActiGraph_official(data)
    if algorithm == ActivityCountAlgorithm.ADS:
        return build_ADS(data)
    if algorithm == ActivityCountAlgorithm.te_Lindert_et_al:
        return build_activity_counts_te_Lindert_et_al(data, axis, prefix)


In [7]:
a = np.array([
    [1, 1, 1],
    [3, 4, 5],
])

In [8]:
np.linalg.norm(a, axis=-1)

array([1.73205081, 7.07106781])

In [9]:
np.linalg.norm(a, axis=0)

array([3.16227766, 4.12310563, 5.09901951])

In [10]:
#| export


def build_ADS(
    time_xyz: np.ndarray,
    sampling_hz: float = 50.0,
    bin_size_seconds: float = 15,
    prefix: str = "",
) -> Tuple[np.ndarray, np.ndarray]:
    """ADS algorithm for activity counts, developed by Arcascope with support from the NHRC.

    Parameters
    ---
     - `time_xyz`: numpy array with shape (N_samples, 4) where the 4 coordinates are: [time, x, y, z] 
     - `sampling_hz`: `float` sampling frequency of thetime_xyz 
    """
    data_shape_error = ValueError(
            f"`time_xyz` must have shape (N_samples, 4) but has shape {time_xyz.shape}"
        )
    try:
        assert (len(time_xyz.shape) == 2 and time_xyz.shape[1] == 4)
    except AssertionError:
        raise data_shape_error

    time_data_raw = time_xyz[:, 0]
    x_accel = time_xyz[:, 1]
    y_accel = time_xyz[:, 2]
    z_accel = time_xyz[:, 3]

    # Interpolate to sampling Hz
    time_values = np.arange(
        np.amin(time_data_raw), np.amax(time_data_raw), 1 / sampling_hz
    )
    # Must do each coordinate separately
    x_data = np.interp(time_values, time_data_raw, x_accel)
    y_data = np.interp(time_values, time_data_raw, y_accel)
    z_data = np.interp(time_values, time_data_raw, z_accel)

    # Calculate "amplitude" = timeseries of 2-norm of (x, y, z)
    amplitude = np.linalg.norm(np.array([x_data, y_data, z_data]), axis = 0)

    abs_amplitude_deriv = np.abs(np.diff(amplitude))
    abs_amplitude_deriv = np.insert(abs_amplitude_deriv, 0, 0)

    # Binning step
    # Sum abs_amplitude_deriv in time-based windows
    # ex: bin_size_seconds = 15
    # Step from first to last time by 15 seconds
    time_counts = np.arange(
        np.amin(time_data_raw), np.amax(time_data_raw), bin_size_seconds
    )

    # Convert time at 50 hz to "# of 15 second windows past start"
    bin_values = (time_values - time_values[0]).astype(int) // bin_size_seconds
    sums_in_bins = np.bincount(bin_values, abs_amplitude_deriv)
    sums_in_bins[sums_in_bins <= 0.05 * max(sums_in_bins)] = 0.0
    return time_counts, sums_in_bins




In [11]:
#| export
from scipy.signal import butter, filtfilt

def build_activity_counts_te_Lindert_et_al(
    time_xyz, axis: int = 3, prefix: str = ""
) -> Tuple[np.ndarray, np.ndarray]:
    """Implementation of the reverse-engineered activity count algorithm from
    te Lindert BH, Van Someren EJ. Sleep. 2013
    Sleep estimates using microelectromechanical systems (MEMS). 
    doi: 10.5665/sleep.2648
    
    :param time_xyz: `np.ndarray` loaded from timestamped triaxial accelerometer CSV. Shape (N, 4)
    :return: (time, activity counts with 15 second epoch)
    """

    # a helper function to calculate max over 2 epochs
    def max2epochs(data, fs, epoch):
        data = data.flatten()

        seconds = int(np.floor(np.shape(data)[0] / fs))
        data = np.abs(data)
        data = data[0 : int(seconds * fs)]

        data = data.reshape(fs, seconds, order="F").copy()

        data = data.max(0)
        data = data.flatten()
        N = np.shape(data)[0]
        num_epochs = int(np.floor(N / epoch))
        data = data[0 : (num_epochs * epoch)]

        data = data.reshape(epoch, num_epochs, order="F").copy()
        epoch_data = np.sum(data, axis=0)
        epoch_data = epoch_data.flatten()

        return epoch_data
    
    fs = 50
    time = np.arange(np.amin(time_xyz[:, 0]), np.amax(time_xyz[:, 0]), 1.0 / fs)
    z_data = np.interp(time, time_xyz[:, 0], time_xyz[:, axis])

    cf_low = 3
    cf_hi = 11
    order = 5
    w1 = cf_low / (fs / 2)
    w2 = cf_hi / (fs / 2)
    pass_band = [w1, w2]
    b, a = butter(order, pass_band, "bandpass")

    z_filt = filtfilt(b, a, z_data)
    z_filt = np.abs(z_filt)

    top_edge = 5
    bottom_edge = 0
    number_of_bins = 128

    bin_edges = np.linspace(bottom_edge, top_edge, number_of_bins + 1)
    binned = np.digitize(z_filt, bin_edges)
    epoch = 15
    counts = max2epochs(binned, fs, epoch)
    counts = (counts - 18) * 3.07
    counts[counts < 0] = 0

    time_counts = np.linspace(np.min(time_xyz[:, 0]), max(time_xyz[:, 0]), np.shape(counts)[0])
    time_counts = np.expand_dims(time_counts, axis=1)
    counts = np.expand_dims(counts, axis=1)

    return time_counts, counts

In [12]:
#| export
from agcounts.extract import get_counts

def build_ActiGraph_official(time_xyz, axis: int = 3) -> Tuple[np.ndarray, np.ndarray]:
    freq = 50
    counts = get_counts(time_xyz[:, 1:], freq=freq, epoch=15)[:, axis - 1]
    times = np.linspace(time_xyz[0, 0], time_xyz[-1, 0], len(counts))

    return times, counts